<img src="../images/JHI_STRAP_Web.png" style="width: 150px; float: right;">
# 04 - Using NCBI `BLAST+` Service  with Biopython

## Learning Outcomes

* Use of `NCBI` remote `BLAST+` using Python and Biopython in the Jupyter notebook
* Creating a remote `BLAST+` query, and its connection to running a `BLAST+` search on the server
* Reading `BLAST+` output from the `NCBI` server into a Python variable
* Using Entrez queries to customise your remote search
* Computational analysis and visualisation of `BLAST+` output

## Table of Contents

1. [Introduction](#introduction)
2. [Python imports](#imports)
3. [Running and analysing a remote BLASTX search](#blastx)
  1. [Run `BLASTX`](#runblastx)
  2. [Load `BLASTX` results](#loadresults)
  3. [Exercise 01](#ex01)

## Introduction

<p></p>
<div class="alert-success">
<b>This notebook presents examples of methods for using `BLAST` programmatically, with the webservice provided by NCBI. All calculations will be run on NCBI's servers, using NCBI's databases (not your local `BLAST` installation), but you will be controlling the search using Python code in this notebook.</b>
</div>

The advantages to using a programmatic interface for remote BLAST queries are the same as those listed in [notebook 03](03-programming_for_blast):

* It is easy to set up repeatable searches for many sequences, or collections of sequences
* It is easy to read in the search results and conduct downstream analyses that add value to your search

Where it is impractical to submit a large number of similar queries via a web form (because it is tiresome to point-and-click over and over again), this can be handled programmatically instead. You have the opportunity to change custom options to help refine your query, just as in the website interface. If you need to repeat a query, it can be trivial to get the same settings every time, if you use a programmatic approach.

<a id="imports"></a>
## Python imports

<p></p>
<div class="alert-success">
<b>To interact with the NCBI's `BLAST` service, we will again use the free `Biopython` programming tools. These provide an interface to interact with NCBI's `BLAST` server, run jobs, and to retrieve the output files.</b>
</div>

We import these tools, and some standard library packages for working with files (`os`) below.

In [1]:
# Show plots as part of the notebook
%matplotlib inline

# Standard library packages
import os

# Import Biopython tools for running remote BLAST searches
from Bio.Blast import NCBIWWW

# Import Biopython SeqIO module to handle reading sequence data
from Bio import SeqIO

<a id="blastx"></a>
## Running and analysing a remote BLASTX search

<p></p>
<div class="alert-success">
<b>As in [notebook 03](03-programming_for_blast.ipynb), your first worked example will be to run a `BLASTX` search, querying a nucleotide sequence against a protein database, to identify potential homologues. What is different about this search is that you will be conducting it at NCBI, and using a different database.</b>
</div>

You will use `Biopython` in the code blocks below to first perform the same `BLASTX` search you carried out in the exercise from [notebook 01](01-blast_at_NCBI_website.ipynb) - this will query a lantibiotic biosynthesis protein from *Kitasatospora* against other *Kitasatospora* species - and write the results to file. Then you will parse the results into a `pandas` dataframe, and finally plot some summary statistics using `seaborn`.

<a id="runblastx"></a>
### Run `BLASTX`

<p></p>
<div class="alert-success">
<b>Running a remote `BLAST` search with `Biopython` is, in some ways, simpler than running a local `BLAST` query. The key steps are:</b>
</div>

1. Read the query sequence(s) from a source (possibly a local file, but maybe a remote database)
2. Run a remote job with the `NCBIWWW.qblast()` method, specifying your query sequence, database, and `BLAST` program
3. Parse the output you get back from NCBI

To run the remote job, you need the same kind of information as if you were running `BLAST` via the web interface - these arguments are compulsory:

* the `BLAST` program to use
* query sequence(s) to search with
* the database to search in

but you can provide some extra choices when you run the remote job, including restricting the remote search on the basis of taxonomy, just as you did in the exercise from `notebook 01` (you will do this later).

<div class="alert-success">
<b>Firstly, you will obtain a query sequence for the search.</b>
</div>

You will use the same penicillin-binding protein from `notebook 01`, reading it as before from a local file, but this time using `Biopython's` `SeqIO()` module.

<p></p>
<div class="alert-warning">
<b>When data such as biological sequences are read in, their metadata - information on database IDs, and other features - follows them. `Biopython` does a nice job of showing us this information if we look at it with the `print()` function:</b>
</div>

In [2]:
# Define path to data directory
datadir = os.path.join("data", "kitasatospora")

# Load sequence of penicillin-binding protein, and inspect the information
seqfile = os.path.join(datadir, "k_sp_CB01950_penicillin.fasta")
seq = SeqIO.read(seqfile, "fasta")
print(seq)

ID: lcl|LISX01000001.1_cds_OKJ16671.1_31
Name: lcl|LISX01000001.1_cds_OKJ16671.1_31
Description: lcl|LISX01000001.1_cds_OKJ16671.1_31 [locus_tag=AMK19_00175] [protein=penicillin-binding protein] [protein_id=OKJ16671.1] [location=39730..41184]
Number of features: 0
Seq('GTGAACAAGCCGATCCGCCGGGTGTCGATCTTCTGCCTGGTCCTGATCCTGGCC...TAG', SingleLetterAlphabet())


<div class="alert-warning">
<b>However, the remote `BLAST` server requires us to present our sequence in `FASTA` format!</b>
</div>

One of the clever things about `Biopython`'s sequence objects - and a big advantage of using programmatic approaches - is that we can readily convert our sequence information into a number of different formats. To do this, we can use the sequence's `.format()` method to produce a `FASTA`-formatted string.

Doing this does not change the original sequence or its information in any way, but it creates a new presentation of that data, which we can use as our query:

In [3]:
# We need the sequence as a string, so use the .format() method
print(seq.format("fasta"))

>lcl|LISX01000001.1_cds_OKJ16671.1_31 [locus_tag=AMK19_00175] [protein=penicillin-binding protein] [protein_id=OKJ16671.1] [location=39730..41184]
GTGAACAAGCCGATCCGCCGGGTGTCGATCTTCTGCCTGGTCCTGATCCTGGCCCTGATG
CTCCGGGTGAACTGGGTGCAGGGCGTTCAGGCGTCGACGTGGGCCAACAACCCGCACAAC
GACCGCACCAAGTACGACAAGTACGCCTACCCGCGCGGCAACATCATCGTCGGCGGCCAG
GCCGTCACCAAGTCCGACTTCGTCAACGGGCTGCGCTACAAGTACAAGCGCTCCTGGGTG
GACGGGCCGATGTACGCGCCGGTCACCGGCTACTCCTCGCAGACGTACGACGCCAGCCAG
CTGGAGAAGCTGGAGGACGGCATCCTCTCCGGCACCGACTCGCGGCTGTTCTTCCGCAAC
ACCCTGGACATGCTGACCGGCAAGCCCAAGCAGGGCGGCGACGTGGTCACCACCATCGAC
CCCAAGGTGCAGAAGGCCGGCTTCGAGGGGCTCGGCAACAAGAAGGGCGCCGCGGTCGCC
ATCGACCCGAAGACCGGGGCGATCCTCGGGCTGGTCTCCACCCCGTCCTACGACCCGGGC
ACCTTCGCGGGCGGCACCAAGGACGACGAGAAGGCCTGGACGGCACTCGACAGCGACCCG
AACAAGCCGATGCTGAACCGGGCGCTGCGCGAGACCTACCCGCCCGGCTCGACCTTCAAG
CTGGTCACCGCGGCGACCGCGTTCGAGACCGGCAAGTACCAGAGCCCGTCGGACGTCACC
GACACCCCGGACCAGTACATCCTGCCCGGCACCAGCACCCCGCTGATCAACGCCAGCCCC
ACCGAGGACTGCGGGAACGCCACCGTGCAACACGCGATGGACCTGTCCTGCAACACGGTG

<div class="alert-success">
<b>You are now almost ready to build your remote `BLAST` query</b>
</div>

The last two things you need to do are to consider the database we're going to search against, and the format you want the data returned in.

* You are going to query against the `pdbaa` database (which contains amino acid sequences for the set of proteins with known structures) as this is small, and should return a result fairly quickly. This is a useful way to identify protein structures that may be useful for threading or other analyses (structural analyses will be covered in session 04).
* You are going to request `"Text"` format output, which will be easy for humans to read.

<p></p>
<div class="alert-warning">
<b>The search will return a special kind of object called a 'handle', which we still need to do some work with, to turn into a file, or read the results from.</b>
</div>

<p></p>
<div class="alert-danger">
<b>NOTE: the remote search may take some time.</b>
</div>

In [4]:
# Run a remote BLASTX search, with the penicillin-binding protein as a query,
# against the pdbaa database, getting text output
result_handle = NCBIWWW.qblast("blastx", "pdbaa", seq.format("fasta"),
                               format_type="Text")

Once the search has completed, you can `read()` the contents of the handle into a variable called `output`, and then inspect the contents using the `print()` function.

In [5]:
# Read the contents of the handle, and print to the cell
output = result_handle.read()
print(output)

<p><!--
QBlastInfoBegin
	Status=READY
QBlastInfoEnd
--><p>
<PRE>
BLASTX 2.8.0+
Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro
A. Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and
David J. Lipman (1997), "Gapped BLAST and PSI-BLAST: a new
generation of protein database search programs", Nucleic
Acids Res. 25:3389-3402.


RID: 95KBPK06014


Database: PDB protein database
           97,152 sequences; 24,542,808 total letters
Query= lcl|LISX01000001.1_cds_OKJ16671.1_31 [locus_tag=AMK19_00175]
[protein=penicillin-binding protein] [protein_id=OKJ16671.1]
[location=39730..41184]

Length=1455


                                                                   Score     E
Sequences producing significant alignments:                       (Bits)  Value

3UN7_A  Chain A, Crystal Structure Of Pbpa From Mycobacterium ...  297     2e-95
3LO7_A  Chain A, Crystal structure of PBPA from Mycobacterium ...  295     2e-94
4MNR_A  Chain A, Crystal Structure Of D,d-transpeptidase Domai... 

<div class="alert-warning">
<h3>QUESTIONS</h3>
<ol>
<li> How is this output different to that from command-line and web-interface `BLAST` results?
<li> How many hits are there?
<li> What is the "best hit" to the query? Do you think it is a 'good' hit?
<li> At what point do you think the matches start to become less reliable? Why do you think this? (*HINT:* inspect the alignments)
</ol>
</div>

<a id="save"></a>
### Save `BLASTX` results to file

The results you obtianed above are human-readable, and similar to the default output type you saw from command-line/terminal `BLAST` in [notebook 02](02-blast_at_terminal.ipynb). But, for now, they exist only in the variable called `output`. If we want to come back to these results at some time in the future, we will need to save them to a file somewhere.

This is a common operation in programmatic approaches to bioinformatics: once a result is obtained, we usually want to save it to a file. Most command-line programs will do this for you, but when working programmatically mostly you will need to save it explicitly, yourself.

The Python code for saving the contents of `output` to the file `output/kitasatospora/remote_blastx_query_01.txt` is given in the cell below:

In [6]:
# Make directory to save output, if it doesn't exist
outdir = os.path.join("output", "kitasatospora")
os.makedirs(outdir, exist_ok=True)

# Save output to file
outfilename = os.path.join(outdir, "remote_blastx_query_01.txt")
with open(outfilename, 'w') as outfh:
    outfh.write(output)

This code does three main things:

1. It creates a variable called `outfilename`, with the path to the file we want to write
2. It opens that file, ready for writing, as a *handle* called `outfh`
3. It writes the contents of `output` into the `outfh` *handle* (and then closes the *handle*)

When this is done, the `BLAST` search results we got from NCBI are written to the file `output/kitasatospora/remote_blastx_query_01.txt`, just as though we did the search locally. You can inspect the contents of that file at the terminal using a command like:

```bash
less output/kitasatospora/remote_blastx_query_01.txt
```

<a id="entrez"></a>
### Using Entrez queries to modify the remote database

The remote NCBI `BLAST+` service provides large, comprehensive databases such as `nr`, `nt` and `refseq`, but it doesn't provide very many specialised databases. Searches against large databases - where you don't care about most of the sequences, and most of the sequences are unlikely to match - can be wasteful and take longer than necessary to get the useful answer you're looking for. But a smaller, specialised organism-centric database - which would be much quicker to search - may not exist separately.

<p></p>
<div class="alert-success">
<b>NCBI provide a service called `Entrez E-Utilities` which allows for complex text-based searches to be defined. A detailed consideration of this very powerful tool is beyond the scope of this course, but we will use the service to replicate the `Organism` field of the NCBI web browser interface that allowed us to restrict the `nr` database only to <i>Kitasatospora</i> sequences.</b>
</div>

In [notebook 01](01-blast_at_NCBI_website.ipynb) you used the `Organism` field in the web browser interface to specify `Kitasatospora (taxid: 2063)` when querying against the complete `nr` database. This had the effect of only querying against sequences in `nr` that originated from *Kitasatospora*, reducing the overall search time and giving you the relevant hits you needed without producing a very large set of output.

To perform a similar filtering of the search using `NCBIWWW.qblast()`, you specify the argument `entrez_query`. The value to be passed is a string that describes the search field you want to filter on. There are many search field options (see the [list here](https://www.ncbi.nlm.nih.gov/books/NBK49540/)), but we will use only one in this lesson: the `organism` field.

<p></p>
<div class="alert-warning">
<b>The `Entrez` query syntax is the same syntax that you would use in any browser-based NCBI database search, and is reported back to you when you refine searches at NCBI. Looking at that output is a good way to find interesting ways to slice large databases into smaller subsets of interest.</b>
</div>

#### The `organism` field `[ORGN]`

The syntax for filtering on organism is `<value>[ORGN]` where `<value>` is the term you want to filter on. This can be the name of the organism, or an identifier such as the NCBI taxon ID.

<p></p>
<div class="alert-warning">
<b>In the cell below, the remote search is conducted on the `nr` database, but restricted only to the set of sequences originating from *Kitasatospora*, by specififying `entrez_query="kitasatospora[ORGN]"`:</b>
</div>

In [7]:
# Repeat the BLAST search on the nr database, restricted to Kitasatospora spp.
result_handle = NCBIWWW.qblast("blastx", "nr", seq.format("fasta"),
                               entrez_query="kitasatospora[ORGN]",
                               format_type="Text")

In [8]:
# Show human-readable results
output = result_handle.read()
print(output)

# Save output to file
outfilename = os.path.join(outdir, "remote_blastx_query_02.txt")
with open(outfilename, 'w') as outfh:
    outfh.write(output)

<p><!--
QBlastInfoBegin
	Status=READY
QBlastInfoEnd
--><p>
<PRE>
BLASTX 2.8.0+
Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro
A. Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and
David J. Lipman (1997), "Gapped BLAST and PSI-BLAST: a new
generation of protein database search programs", Nucleic
Acids Res. 25:3389-3402.


RID: 95KBUH39015


Database: All non-redundant GenBank CDS
translations+PDB+SwissProt+PIR+PRF excluding environmental samples
from WGS projects
           146,786,844 sequences; 53,795,819,935 total letters
Query= lcl|LISX01000001.1_cds_OKJ16671.1_31 [locus_tag=AMK19_00175]
[protein=penicillin-binding protein] [protein_id=OKJ16671.1]
[location=39730..41184]

Length=1455


                                                                   Score     E
Sequences producing significant alignments:                       (Bits)  Value

WP_073807071.1  penicillin-binding protein 2 [Kitasatospora sp...  996     0.0   
WP_035864460.1  penicillin-binding protein 

<div class="alert-warning">
<h3>QUESTIONS</h3>
<ol>
<li> How does this output differ from that of the same search conducted through the web browser interface?
</ol>
</div>

<img src="../images/exercise.png" style="width: 100px; float: left;">
<a id="ex01"></a>
### Exercise 01 (10min)

Using the sequence in the file `data/kitasatospora/lantibiotic.fasta` as a query, can you do the following?

<p></p>
<div class="alert-danger">
<ul>
<li> Conduct a remote `BLASTX` query against the `nr` database, restricted to <i>Kitasatospora</i> proteins
<li> Write the output to a new file called `lantibiotic_blastx_nr_kitasatospora.txt` in the output directory `output/kitasatospora`.
<li> Compare the results to the output from Exercise 01 in `notebook 03`.
</ul>
</div>

In [9]:
# SOLUTION - EXERCISE 01

# Load sequence of lantibiotic synthesis protein, and inspect the information
seq = SeqIO.read("data/kitasatospora/lantibiotic.fasta", "fasta")

# Run the BLASTX search on the nr database, restricted to Kitasatospora spp.
result_handle = NCBIWWW.qblast("blastx", "nr", seq.format("fasta"),
                               entrez_query="kitasatospora[ORGN]",
                               format_type="Text")

# Save output to file
outfilename = os.path.join('output', 'kitasatospora', 'lantibiotic_blastx_nr_kitasatospora.txt')
with open(outfilename, 'w') as outfh:
    outfh.write(result_handle.read())